In [ ]:
import pandas as pd
import sqlite3
import random
from sqlite3 import Error
import hashlib

In [ ]:
sales = pd.read_excel('AW_Sales_task.xlsx')

In [ ]:
pd.to_datetime(sales['OrderDate'])

In [ ]:
sales.info()

In [ ]:
sales.head()

In [ ]:
sales.columns

In [ ]:
#rename columns for match to exercise :
sales['Province'] = sales['Territory']
sales['ProvinceID'] = sales['TerritoryID']
sales['AgentID'] = sales['SalesPersonID']
sales['Agency'] = sales['TerritoryGroup']
sales['CustomerName'] = sales['CustomerID']

In [ ]:
sales.to_csv('sales_1')

In [ ]:
discount_codes = ['DISC10', 'DISC20', 'NONE', None]

In [ ]:
def hash_customer_id(customer_id):
    return int(hashlib.sha256(str(customer_id).encode('utf-8')).hexdigest(), 16)

In [ ]:
def generate_customer_name(customer_id):
    # Lists of potential first and last names
    first_names = ['John', 'Jane', 'Chris', 'Kathy', 'Mike', 'Nancy', 'Alex', 'Mary', 'Steve', 'Linda']
    last_names = ['Smith', 'Johnson', 'Williams', 'Jones', 'Brown', 'Davis', 'Miller', 'Wilson', 'Moore', 'Taylor']
    
    # Seed the random number generator with a hash of the customer ID
    seed = hash_customer_id(customer_id)
    random.seed(seed)
    
    # Generate a random first name and last name
    first_name = random.choice(first_names)
    last_name = random.choice(last_names)
    
    # Return the combined name
    return f"{first_name} {last_name}"

In [ ]:
unique_customer_ids = sales['CustomerID'].unique()

# Generate a name for each unique customer ID
customer_names = {customer_id: generate_customer_name(customer_id) for customer_id in unique_customer_ids}

# Map the generated names back to the DataFrame
sales['CustomerName'] = sales['CustomerID'].map(customer_names)

In [ ]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

def execute_sql(conn, sql, data=None):
    """ Execute SQL statement """
    try:
        c = conn.cursor()
        if data is not None:
            c.execute(sql, data)
        else:
            c.execute(sql)
        conn.commit()
        
    except Error as e:
        print(e)

if __name__ == '__main__':
    database = "strasport_2.db"
    
    sql_create_globalTabel = """CREATE TABLE IF NOT EXISTS SalesData (
    ID INTEGER PRIMARY KEY,
    OrderDetailID INTEGER,
    OrderDate TEXT,
    DueDate TEXT,
    ShipDate TEXT,
    StatusID INTEGER,
    Status TEXT,
    CustomerID INTEGER,
    SalesPersonID INTEGER,
    TerritoryID INTEGER,
    Territory TEXT,
    TerritoryGroup TEXT,
    ShipMethodID INTEGER,
    ShipMethod TEXT,
    ProductID INTEGER,
    Product TEXT,
    ProductSubCategory TEXT,
    ProductCategory TEXT,
    OrderQty INTEGER,
    UnitPrice REAL,
    LineTotal REAL,
    TaxAmt REAL,
    TotalDue REAL,
    Province TEXT,
    ProvinceID INTEGER,
    AgentID INTEGER,
    Agency TEXT,
    CustomerName TEXT
);"""

    sql_create_store_table = """ CREATE TABLE IF NOT EXISTS Store (
                                        TerritoryID INTEGER PRIMARY KEY,
                                        Territory TEXT NOT NULL,
                                        TerritoryGroup TEXT NOT NULL
                                    ); """

    sql_create_client_table = """CREATE TABLE IF NOT EXISTS Client (
                                    ClientID INTEGER PRIMARY KEY,
                                    ClientName TEXT NOT NULL
                                );"""

    sql_create_product_table = """CREATE TABLE IF NOT EXISTS Product (
                                    ProductID INTEGER PRIMARY KEY,
                                    ProductType TEXT NOT NULL
                                );"""

    sql_create_transactions_table = """ CREATE TABLE IF NOT EXISTS Transactions (
                                        TransactionID INTEGER PRIMARY KEY,
                                        StoreID INTEGER NOT NULL,
                                        ClientID INTEGER NOT NULL,
                                        ProductID INTEGER NOT NULL,
                                        TransactionDate INTEGER,
                                        QuantitySold INTEGER NOT NULL,
                                        DiscountCode TEXT,
                                        FOREIGN KEY (StoreID) REFERENCES Store(TerritoryID),
                                        FOREIGN KEY (ClientID) REFERENCES Client (ClientID),
                                        FOREIGN KEY (ProductID) REFERENCES Product (ProductID),
                                        FOREIGN KEY (TransactionDate) REFERENCES DimTime(DateKey)

                                    ); """
    
    sql_create_DimensionTable = """ CREATE TABLE DimTime (
                                    DateKey INTEGER PRIMARY KEY,
                                    Day INTEGER,
                                    Month INTEGER,
                                    Quarter INTEGER,
                                    Semester INTEGER,
                                    Year INTEGER
                                ); """
    

    sql_Fqct_trunover = """CREATE TABLE FactTurnover (
    TurnoverID INTEGER PRIMARY KEY,
    DateKey INTEGER,
    StoreID INTEGER,
    ClientID INTEGER,
    AmountBilled DECIMAL,
    QuantityInvoiced INTEGER,
    DiscountValue DECIMAL,
    FOREIGN KEY (DateKey) REFERENCES DimTime(DateKey),
    FOREIGN KEY (StoreID) REFERENCES Store(TerritoryID),
    FOREIGN KEY (ClientID) REFERENCES Client ( ClientID)
);"""

    
    conn = sqlite3.connect(database)

   
    if conn is not None:
        execute_sql(conn, sql_create_store_table)
        execute_sql(conn, sql_create_client_table)
        execute_sql(conn, sql_create_product_table)
        execute_sql(conn, sql_create_transactions_table)
        execute_sql(conn, sql_create_DimensionTable)
        execute_sql(conn, sql_Fqct_trunover)
        
    else:
        print("Error! cannot create the database connection.")


In [ ]:
#insert all the data in raw inside the database, but need to chunk it becasue too much lines
def insert_in_chunks(dataframe, chunk_size=100):
    for start_row in range(0, dataframe.shape[0], chunk_size):
        end_row = start_row + chunk_size
        chunk = dataframe[start_row:end_row]
        chunk.to_sql('SalesData', conn, if_exists='append', index=False, method=None)


insert_in_chunks(sales, 100)

In [ ]:
#create def for insert data inside the table 
def insert_stores(conn, store):
    
    sql = ''' INSERT INTO Store(TerritoryID, Territory, TerritoryGroup)
              VALUES(?,?,?) '''
    execute_sql(conn, sql, store)
    

def insert_client(conn, client):
   
    sql = ''' INSERT INTO Client(ClientID, ClientName)
              VALUES(?,?) '''
    execute_sql(conn, sql, client)

def insert_product(conn, product):
    
    sql = ''' INSERT INTO Product(ProductID, ProductType)
              VALUES(?,?) '''
    execute_sql(conn, sql, product)

def insert_transaction(conn, transaction):
    
    sql = ''' INSERT INTO Transactions(TransactionID, StoreID, ClientID, ProductID, TransactionDate, QuantitySold, DiscountCode)
              VALUES(?,?,?,?,?,?,?) '''
    execute_sql(conn, sql, transaction)


In [ ]:
unique_clients = sales[['CustomerID', 'CustomerName']].drop_duplicates()
for _, row in unique_clients.iterrows():
    insert_client(conn, (row['CustomerID'], row['CustomerName']))


In [ ]:
for _, row in sales.iterrows():
    stores = (row['TerritoryID'], row['Territory'],row['TerritoryGroup'])
    insert_stores(conn, stores)

In [ ]:
unique_products = sales[['ProductID', 'ProductSubCategory']].drop_duplicates()
for _, row in unique_products.iterrows():
    insert_product(conn, (row['ProductID'], row['ProductSubCategory']))


In [ ]:
sales['OrderDate'] = pd.to_datetime(sales['OrderDate'])

for _, row in sales.iterrows():
   
    order_date_str = row['OrderDate'].strftime('%Y-%m-%d %H:%M:%S')
    
    transaction = (
        row['OrderDetailID'],  
        row['AgentID'],       
        row['CustomerID'],
        row['ProductID'],
        order_date_str,       
        row['OrderQty'],       
        row.get('DiscountCode', None) 
    )
    insert_transaction(conn, transaction)


In [ ]:
sales.columns

In [ ]:
if conn:
    conn.close()